In [1]:
import re
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#from sklearn.neighbors import NearestNeighbors
#from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

In [2]:
bt = pd.read_csv('data_d_kor.csv')
NaN_processing_bt = bt.fillna(0) 
indexing_bt = NaN_processing_bt.set_index('date') 
print(indexing_bt)
btusa = pd.read_csv('data_d_usa.csv')
NaN_processing_btusa = btusa.fillna(0) 
indexing_bt_usa = NaN_processing_btusa.set_index('date') 

print(indexing_bt_usa)

global list_for_Cal
global list_for_AW_usa

list_for_AW_kor =[]
list_for_AW_usa =[]

              gold  commodity   k10bond   kospi
date                                           
2000-01-03  0.0000    -0.0016  0.000000  0.0000
2000-01-04 -0.0201    -0.0096  0.000000  0.0301
2000-01-05 -0.0057    -0.0096  0.000000 -0.0687
2000-01-06  0.0011    -0.0005  0.000000 -0.0259
2000-01-07  0.0018    -0.0098  0.000000 -0.0126
...            ...        ...       ...     ...
2022-07-25 -0.0048     0.0141  0.000124  0.0044
2022-07-26 -0.0008     0.0023  0.000122  0.0039
2022-07-27  0.0008     0.0143  0.000120  0.0011
2022-07-28  0.0195     0.0000  0.000123  0.0082
2022-07-29  0.0000     0.0000  0.000000  0.0000

[6786 rows x 4 columns]
              gold  commodity  snp500   u10bond
date                                           
2000-01-03  0.0000    -0.0016 -0.0095  0.000000
2000-01-04 -0.0201    -0.0096 -0.0383  0.000250
2000-01-05 -0.0057    -0.0096  0.0019  0.000254
2000-01-06  0.0011    -0.0005  0.0010  0.000251
2000-01-07  0.0018    -0.0098  0.0271  0.000250
...            

In [3]:
def Select_Date(Start, End, Dataframe) :   # 구간 데이터 모으는 함수
    """
    gold = gold.loc['{}'.format(Start):'{}'.format(End), 'gold']
    u10bond = u10bond.loc['{}'.format(Start):'{}'.format(End),'u10bond']
    snp500= snp500.loc['{}'.format(Start):'{}'.format(End) ,'snp500']
    commodity= commodity.loc['{}'.format(Start):'{}'.format(End), 'commodity']
    """
    total = indexing_bt.loc['{}'.format(Start):'{}'.format(End)]

    Dataframe = pd.concat([Dataframe,total]) # 리스트형식.. tqtqtqtqtqtq
    return Dataframe

def Select_Date_usa(Start, End, Dataframe) :   # 구간 데이터 모으는 함수
    
    total = indexing_bt_usa.loc['{}'.format(Start):'{}'.format(End)]

    Dataframe = pd.concat([Dataframe,total]) # 리스트형식.. tqtqtqtqtqtq
    return Dataframe





def Analysis(Dataframe):


    M = 10 # 이웃 몇개 채취할거임?

    x = Dataframe.iloc[:, 2]  # 샤프
    x = x.values.reshape(len(Dataframe),1) # fit 시키기 위해 변형

    neighbor = NearestNeighbors(n_neighbors=M, radius=0.4)
    neighbor.fit(x)


    nparray1 = neighbor.kneighbors(Dataframe.iloc[Dataframe.idxmax()[2], 2].reshape(-1, 1), M, return_distance=False)
    #print(Dataframe.loc[Dataframe.idxmax()[2]])
    #print(Dataframe.iloc[nparray1[0]])

    kospi = 0
    k10bond = 0
    gold = 0
    commodity = 0


    for i in range(0, M):

        kospi += Dataframe.loc[nparray1[0][i], 'kospi']
        k10bond += Dataframe.loc[nparray1[0][i], 'k10bond']
        gold += Dataframe.loc[nparray1[0][i], 'gold']
        commodity += Dataframe.loc[nparray1[0][i], 'commodity']

    print("\n높은 Sharpe 지수를 가진 상위 {}개의 포트폴리오를 샘플링하여 평균낸 결과 \n시장 지수 추종 주식 {}%, 국채 10년 {}%, 금 {}%, 원자재 {}% 포트폴리오가 제안되었습니다. ".format(M, kospi/M, k10bond/M, gold/M, commodity/M))

    print("\n가장 높은 Sharpe 지수를 보인 포트폴리오는 아래와 같습니다. ")
    print(Dataframe.loc[Dataframe.idxmax()[2]])



def Scatter(Dataframe):

    # scatter
    plt.subplots(figsize=[10, 10])
    scatter1 = plt.scatter(Dataframe['Volatility'], Dataframe['Return %'], marker='o', s=10, alpha=0.5) #알파는 투명도


#    cursor1 = mplcursors.cursor(scatter1, hover=True)
#    cursor1.connect("add", lambda sel: sel.annotation.set_text('Portfolio \n Index : {}\n Return : {}%\n Volatility : {}\n Sharpe : {}\n MDD : {}%\n snp500 : {}\n u10bond : {}\n k5bond : {}\n gold : {}\n commodity : {}'.format(sel.target.index, sel.target[1], sel.target[0], Dataframe.iloc[sel.target.index, 2], Dataframe.iloc[sel.target.index, 1], Dataframe.iloc[sel.target.index, 4], Dataframe.iloc[sel.target.index, 5], Dataframe.iloc[sel.target.index, 6], Dataframe.iloc[sel.target.index, 7], Dataframe.iloc[sel.target.index, 8])))


    plt.scatter(Dataframe.iloc[Dataframe.idxmax()[2], 3], Dataframe.iloc[Dataframe.idxmax()[2], 0], marker='x', c='red', s=15)
    plt.annotate('Sharpe Max', fontsize=10, xy=(Dataframe.iloc[Dataframe.idxmax()[2], 3], Dataframe.iloc[Dataframe.idxmax()[2], 0]), xytext=(Dataframe.iloc[Dataframe.idxmax()[2], 3] -0.03 , Dataframe.iloc[Dataframe.idxmax()[2], 0]+0.15))

    plt.scatter(Dataframe.iloc[Dataframe.idxmin()[3], 3], Dataframe.iloc[Dataframe.idxmin()[3], 0], marker='x', c='red',s=15)
    plt.annotate('Volatility Min', fontsize=10, xy=(Dataframe.iloc[Dataframe.idxmin()[3], 3], Dataframe.iloc[Dataframe.idxmin()[3], 0]),
                 xytext=(Dataframe.iloc[Dataframe.idxmin()[3], 3] - 0.03, Dataframe.iloc[Dataframe.idxmin()[3], 0] + 0.15), )

    plt.scatter(Dataframe.iloc[Dataframe.idxmax()[0], 3], Dataframe.iloc[Dataframe.idxmax()[0], 0], marker='x', c='red',
                s=15)
    plt.annotate('Return Max', fontsize=10,
                 xy=(Dataframe.iloc[Dataframe.idxmax()[0], 3], Dataframe.iloc[Dataframe.idxmax()[0], 0]), xytext=(
        Dataframe.iloc[Dataframe.idxmax()[0], 3] - 0.03, Dataframe.iloc[Dataframe.idxmax()[0], 0] + 0.15))

    plt.scatter(Dataframe.iloc[Dataframe.idxmax()[1], 3], Dataframe.iloc[Dataframe.idxmax()[1], 0], marker='x', c='red',
                s=15)
    plt.annotate('MDD Min', fontsize=10,
                 xy=(Dataframe.iloc[Dataframe.idxmax()[1], 3], Dataframe.iloc[Dataframe.idxmax()[1], 0]), xytext=(
        Dataframe.iloc[Dataframe.idxmax()[1], 3] - 0.03, Dataframe.iloc[Dataframe.idxmax()[1], 0] + 0.15))

    #람다식

    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.show()






In [4]:
def Backtesting_manualset_kor(a, b, c, d, Data):

        stock = a
        bond = b
        gold = c
        commodity = d

        list_for_percent = []
        list_for_var = []
        global list_for_AW_kor

        R = None

        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i])  
            bond = bond * (1 + Data['k10bond'][i])
            stock = stock * (1 + Data['kospi'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_var = gold + bond + stock + commodity
            list_for_percent.append(sum_for_var - 100)

            if R:  
                r = (sum_for_var - R) / R * 100
                list_for_var.append(r)
            R = sum_for_var

        rate_of_return = (gold + bond + stock + commodity - 100)  
        list_for_AW_kor=list_for_var
        var_portfolio = np.array(list_for_var).var()  
        
   
        Volatility_standard_deviation = np.sqrt(var_portfolio)
        Sharpe_ratio = (rate_of_return / 100) / Volatility_standard_deviation  
        return_dataframe = pd.DataFrame(
            {'Return %': [round(rate_of_return, 4)], 'Sharpe': [round(Sharpe_ratio, 4)],
             'Volatility': [round(Volatility_standard_deviation, 4)]})


        


        return list_for_percent
    
def Backtesting_manualset_usa(a, b, c, d, Data):

        stock = a
        bond = b
        gold = c
        commodity = d

        list_for_percent = []
        list_for_var = []
        global list_for_AW_usa

        R = None

        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i])  
            bond = bond * (1 + Data['u10bond'][i])
            stock = stock * (1 + Data['snp500'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_var = gold + bond + stock + commodity
            list_for_percent.append(sum_for_var - 100)

            if R:  
                r = (sum_for_var - R) / R * 100
                list_for_var.append(r)
            R = sum_for_var

        rate_of_return = (gold + bond + stock + commodity - 100)  
        list_for_AW_usa=list_for_var
        var_portfolio = np.array(list_for_var).var()  
        Volatility_standard_deviation = np.sqrt(var_portfolio)
        Sharpe_ratio = (rate_of_return / 100) / Volatility_standard_deviation  
        return_dataframe = pd.DataFrame(
            {'Return %': [round(rate_of_return, 4)], 'Sharpe': [round(Sharpe_ratio, 4)],
             'Volatility': [round(Volatility_standard_deviation, 4)]})



        return list_for_percent

In [5]:
Colected_Data = pd.DataFrame()
Colected_Data_usa = pd.DataFrame()
array_for_sort = pd.DataFrame()
all_array_for_sort = pd.DataFrame()

#백테스팅 날짜 선택하세요 xxxx-xx 형식 지킬것

Colected_Data = Select_Date('2008-02', '2008-03', Colected_Data) 
Colected_Data = Select_Date('2013-03', '2014-09', Colected_Data) 
#Colected_Data = Select_Date('2018-06', '2018-10', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2008-02', '2008-03', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2013-03', '2014-09', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2018-06', '2018-10', Colected_Data_usa) 


"""
usa(U) kor(K) classification

K == 2 and U == 1 : 
Colected_Data = Select_Date('2000-09', '2000-10', Colected_Data) 
Colected_Data = Select_Date('2010-09', '2011-06', Colected_Data) 
Colected_Data = Select_Date('2015-06', '2016-09', Colected_Data) 
#Colected_Data = Select_Date('2020-11', '2020-12', Colected_Data) 
#Colected_Data = Select_Date('2021-12', '2022-01', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2000-09', '2000-10', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2010-09', '2011-06', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2015-06', '2016-09', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2020-11', '2020-12', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2021-12', '2022-01', Colected_Data_usa) 



K == 1 and U == 1 :
Colected_Data = Select_Date('2000-11', '2000-12', Colected_Data) 
Colected_Data = Select_Date('2003-08', '2003-10', Colected_Data) 
Colected_Data = Select_Date('2011-07', '2013-02', Colected_Data) 
#Colected_Data = Select_Date('2018-11', '2020-01', Colected_Data) 
#Colected_Data = Select_Date('2022-02', '2022-07', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2000-11', '2000-12', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2003-08', '2003-10', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2011-07', '2013-02', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2018-11', '2020-01', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2022-02', '2022-07', Colected_Data_usa) 




K == 1 and U == 0 :
Colected_Data = Select_Date('2001-10', '2001-11', Colected_Data) 
Colected_Data = Select_Date('2003-01', '2003-07', Colected_Data) 
Colected_Data = Select_Date('2009-02', '2009-03', Colected_Data) 
#Colected_Data = Select_Date('2020-02', '2020-08', Colected_Data)
Colected_Data_usa = Select_Date_usa('2001-10', '2001-11', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2003-01', '2003-07', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2009-02', '2009-03', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2020-02', '2020-08', Colected_Data_usa) 


K == 0 and U == 1 :
Colected_Data = Select_Date('2008-07', '2008-07', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2008-07', '2008-07', Colected_Data_usa) 


K == 2 and U == 2 :
Colected_Data = Select_Date('2003-11', '2006-09', Colected_Data) 
Colected_Data = Select_Date('2014-10', '2015-05', Colected_Data) 
#Colected_Data = Select_Date('2017-10', '2018-05', Colected_Data) 
#Colected_Data = Select_Date('2021-09', '2021-11', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2003-11', '2006-09', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2014-10', '2015-05', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2017-10', '2018-05', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2021-09', '2021-11', Colected_Data_usa) 




K == 3 and U == 2 :

Colected_Data = Select_Date('2000-01', '2000-08', Colected_Data) 
#Colected_Data = Select_Date('2021-03', '2021-08', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2000-01', '2000-08', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2021-03', '2021-08', Colected_Data_usa) 


K == 3 and U == 1 :

Colected_Data = Select_Date('2002-02', '2002-06', Colected_Data) 
Colected_Data = Select_Date('2010-01', '2010-08', Colected_Data) 
Colected_Data = Select_Date('2016-10', '2016-12', Colected_Data) #16년 말까지만.  
#Colected_Data = Select_Date('2016-10', '2017-09', Colected_Data) 
#Colected_Data = Select_Date('2021-01', '2021-02', Colected_Data)
Colected_Data_usa = Select_Date_usa('2002-02', '2002-06', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2010-01', '2010-08', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2016-10', '2016-12', Colected_Data_usa) #16년 말까지만.  
#Colected_Data_usa = Select_Date_usa('2016-10', '2017-09', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2021-01', '2021-02', Colected_Data_usa) 


K == 2 and U == 0 :

Colected_Data = Select_Date('2001-12', '2002-01', Colected_Data) 
Colected_Data = Select_Date('2002-10', '2002-12', Colected_Data) 
Colected_Data = Select_Date('2009-04', '2009-05', Colected_Data) 
#Colected_Data = Select_Date('2020-09', '2020-10', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2001-12', '2002-01', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2002-10', '2002-12', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2009-04', '2009-05', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2020-09', '2020-10', Colected_Data_usa) 

K == 3 and U == 0 :

Colected_Data = Select_Date('2002-07', '2002-09', Colected_Data) 
Colected_Data = Select_Date('2009-06', '2009-12', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2002-07', '2002-09', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2009-06', '2009-12', Colected_Data_usa) 





K == 0 and U == 2 :
Colected_Data = Select_Date('2008-04', '2008-06', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2008-04', '2008-06', Colected_Data_usa) 

K == 0 and U == 0 :

Colected_Data = Select_Date('2001-01', '2001-09', Colected_Data) 
Colected_Data = Select_Date('2008-08', '2009-01', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2001-01', '2001-09', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2008-08', '2009-01', Colected_Data_usa) 


K == 2 and U == 3 :

Colected_Data = Select_Date('2006-10', '2007-12', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2006-10', '2007-12', Colected_Data_usa) 

K == 1 and U == 3 :
Colected_Data = Select_Date('2008-01', '2008-01', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2008-01', '2008-01', Colected_Data_usa) 

K == 1 and U == 2 :
Colected_Data = Select_Date('2008-02', '2008-03', Colected_Data) 
Colected_Data = Select_Date('2013-03', '2014-09', Colected_Data) 
#Colected_Data = Select_Date('2018-06', '2018-10', Colected_Data) 
Colected_Data_usa = Select_Date_usa('2008-02', '2008-03', Colected_Data_usa) 
Colected_Data_usa = Select_Date_usa('2013-03', '2014-09', Colected_Data_usa) 
#Colected_Data_usa = Select_Date_usa('2018-06', '2018-10', Colected_Data_usa) 

"""






"\nusa(U) kor(K) classification\n\nK == 2 and U == 1 : \nColected_Data = Select_Date('2000-09', '2000-10', Colected_Data) \nColected_Data = Select_Date('2010-09', '2011-06', Colected_Data) \nColected_Data = Select_Date('2015-06', '2016-09', Colected_Data) \n#Colected_Data = Select_Date('2020-11', '2020-12', Colected_Data) \n#Colected_Data = Select_Date('2021-12', '2022-01', Colected_Data) \nColected_Data_usa = Select_Date_usa('2000-09', '2000-10', Colected_Data_usa) \nColected_Data_usa = Select_Date_usa('2010-09', '2011-06', Colected_Data_usa) \nColected_Data_usa = Select_Date_usa('2015-06', '2016-09', Colected_Data_usa) \n#Colected_Data_usa = Select_Date_usa('2020-11', '2020-12', Colected_Data_usa) \n#Colected_Data_usa = Select_Date_usa('2021-12', '2022-01', Colected_Data_usa) \n\n\n\nK == 1 and U == 1 :\nColected_Data = Select_Date('2000-11', '2000-12', Colected_Data) \nColected_Data = Select_Date('2003-08', '2003-10', Colected_Data) \nColected_Data = Select_Date('2011-07', '2013-02'

In [6]:



'''
korweight_0 = pd.DataFrame(Backtesting_manualset_kor(10, 30, 50, 10, Colected_Data)) #0
korweight_1 = pd.DataFrame(Backtesting_manualset_kor(30, 50, 10, 10, Colected_Data)) #1
korweight_2 = pd.DataFrame(Backtesting_manualset_kor(24, 50, 16, 10, Colected_Data)) #2
korweight_3 = pd.DataFrame(Backtesting_manualset_kor(10, 30, 10, 50, Colected_Data)) #3

usaweight_0 = pd.DataFrame(Backtesting_manualset_usa(10, 50, 30, 10, Colected_Data_usa)) #0
usaweight_1 = pd.DataFrame(Backtesting_manualset_usa(12, 50, 28, 10, Colected_Data_usa)) #1
usaweight_2 = pd.DataFrame(Backtesting_manualset_usa(30, 50, 10, 10, Colected_Data_usa)) #2
usaweight_3 = pd.DataFrame(Backtesting_manualset_usa(10, 50, 30, 10, Colected_Data_usa)) #3

'''



'\nkorweight_0 = pd.DataFrame(Backtesting_manualset_kor(10, 30, 50, 10, Colected_Data)) #0\nkorweight_1 = pd.DataFrame(Backtesting_manualset_kor(30, 50, 10, 10, Colected_Data)) #1\nkorweight_2 = pd.DataFrame(Backtesting_manualset_kor(24, 50, 16, 10, Colected_Data)) #2\nkorweight_3 = pd.DataFrame(Backtesting_manualset_kor(10, 30, 10, 50, Colected_Data)) #3\n\nusaweight_0 = pd.DataFrame(Backtesting_manualset_usa(10, 50, 30, 10, Colected_Data_usa)) #0\nusaweight_1 = pd.DataFrame(Backtesting_manualset_usa(12, 50, 28, 10, Colected_Data_usa)) #1\nusaweight_2 = pd.DataFrame(Backtesting_manualset_usa(30, 50, 10, 10, Colected_Data_usa)) #2\nusaweight_3 = pd.DataFrame(Backtesting_manualset_usa(10, 50, 30, 10, Colected_Data_usa)) #3\n\n'

In [7]:

korweight = pd.DataFrame(Backtesting_manualset_kor(20, 60, 10, 10, Colected_Data)) #1
usaweight = pd.DataFrame(Backtesting_manualset_usa(20, 60, 10, 10, Colected_Data_usa)) #2



for a in range(0,11):
    array_AW_var = [list_for_AW_kor[i]*a*0.1 + 0.1*(10-a)*list_for_AW_usa[i] for i in range(len(list_for_AW_kor))]
    varAW = np.array(array_AW_var).var()
    standard_deviationAW = np.sqrt(varAW)    
    
    print("올웨더 전략 KOR:USA = {}:{} 의 결과 ".format(a, 10-a))
                                                 
    print('Return %: {}, Sharpe: {}, Volatility: {}\n'.format(round((korweight.iloc[-1]), 4)[0]*a*0.1 + 0.1*(10-a)*round((usaweight.iloc[-1]), 4)[0],
                                                              round((round((korweight.iloc[-1]) , 4)[0]*a*0.1 + 0.1*(10-a)*round((usaweight.iloc[-1]) , 4)[0])/100/standard_deviationAW, 4), round(standard_deviationAW, 4)))  
   

    print('---------------------------------------')      
    
    





올웨더 전략 KOR:USA = 0:10 의 결과 
Return %: 7.551, Sharpe: 0.3352, Volatility: 0.2253

---------------------------------------
올웨더 전략 KOR:USA = 1:9 의 결과 
Return %: 7.198340000000001, Sharpe: 0.3304, Volatility: 0.2179

---------------------------------------
올웨더 전략 KOR:USA = 2:8 의 결과 
Return %: 6.845680000000001, Sharpe: 0.3228, Volatility: 0.2121

---------------------------------------
올웨더 전략 KOR:USA = 3:7 의 결과 
Return %: 6.4930200000000005, Sharpe: 0.3123, Volatility: 0.2079

---------------------------------------
올웨더 전략 KOR:USA = 4:6 의 결과 
Return %: 6.140360000000001, Sharpe: 0.2988, Volatility: 0.2055

---------------------------------------
올웨더 전략 KOR:USA = 5:5 의 결과 
Return %: 5.7877, Sharpe: 0.2825, Volatility: 0.2049

---------------------------------------
올웨더 전략 KOR:USA = 6:4 의 결과 
Return %: 5.435040000000001, Sharpe: 0.2636, Volatility: 0.2062

---------------------------------------
올웨더 전략 KOR:USA = 7:3 의 결과 
Return %: 5.082380000000001, Sharpe: 0.2429, Volatility: 0.2092

-----